In [ ]:
!aws s3 ls s3://sagemaker-us-east-1-441707470931/data/tf-distribution-options-index/

In [1]:
import utilities
import tensorflow as tf
from numpy.random import seed
seed(13)
from tensorflow import set_random_seed
set_random_seed(13)

In [2]:
from model_def import HEIGHT, WIDTH, DEPTH, NUM_CLASSES
from utilities import _train_preprocess_fn

def _dataset_parser(value):

    featdef = {
        'image': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64)
    #    'index': tf.FixedLenFeature([], tf.int64)
    }

    example = tf.parse_single_example(value, featdef)
    image = tf.decode_raw(example['image'], tf.uint8)
    image.set_shape([DEPTH * HEIGHT * WIDTH])

    # Reshape from [depth * height * width] to [depth, height, width].
    image = tf.cast(
        tf.transpose(tf.reshape(image, [DEPTH, HEIGHT, WIDTH]), [1, 2, 0]),
        tf.float32)
    label = tf.cast(example['label'], tf.int32)
    #index = tf.cast(example['index'], tf.int32)
    image = _train_preprocess_fn(image)
    return image, tf.one_hot(label, NUM_CLASSES)#, index

In [3]:
def get_dataset(channel, path):
    if channel == 'train':
        records = [48,62,51,15,91,95,88,11,22,33,3,77,55,26,73,59,40,80,19,99,44,7,66,37,84]
        filenames = ['../data/train/train_{}.tfrecords'.format(num) for num in records]
    else:
        filenames = utilities._get_filenames(channel, path)
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.repeat()
    dataset = dataset.prefetch(10)
    dataset = dataset.map(_dataset_parser, num_parallel_calls=10)
    dataset = dataset.batch(128, drop_remainder=True)
    if channel == 'train':
        # Ensure that the capacity is sufficiently large to provide good random
        # shuffling.
        buffer_size = int(50000 * 0.4) + 3 * 128
        dataset = dataset.shuffle(buffer_size=buffer_size)
    return dataset

In [4]:
from model_def import get_model
model = get_model(0.001, 2e-4, 'adam', 0.9, 1)

W0822 03:19:10.765670 139964965705536 deprecation.py:506] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
records = [6,41,2,0,30,63,38,27,74,96,78,92,16,67,70,81,56,89,34,45,49,52,12,23,85]

In [6]:
train_dataset = get_dataset('train', '../data/train/')
val_dataset = get_dataset('validation', '../data/validation')

W0822 03:19:12.531327 139964965705536 deprecation_wrapper.py:119] From /home/ec2-user/SageMaker/amazon-sagemaker-script-mode/tf-distribution-options/code/utilities.py:25: The name tf.image.resize_image_with_crop_or_pad is deprecated. Please use tf.image.resize_with_crop_or_pad instead.

W0822 03:19:12.543909 139964965705536 deprecation_wrapper.py:119] From /home/ec2-user/SageMaker/amazon-sagemaker-script-mode/tf-distribution-options/code/utilities.py:28: The name tf.random_crop is deprecated. Please use tf.image.random_crop instead.

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:10: UserWarning: Seed 13 from outer graph might be getting used by function Dataset_map__dataset_parser, if the random op has not been provided any seed. Explicitly set the seed in the function if this is not the intended behavior.
W0822 03:19:12.589290 139964965705536 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packag

In [7]:
model.fit(train_dataset, epochs=100, steps_per_epoch=10000//128, validation_steps=10000//128, validation_data=val_dataset)

Epoch 1/100
78/78 [==============================] - 322s 4s/step - loss: 2.2249 - acc: 0.2216 - val_loss: 1.9895 - val_acc: 0.2470
Epoch 2/100
78/78 [==============================] - 39s 505ms/step - loss: 1.8061 - acc: 0.3197 - val_loss: 2.2591 - val_acc: 0.2065
Epoch 3/100
78/78 [==============================] - 44s 569ms/step - loss: 1.6929 - acc: 0.3653 - val_loss: 1.7996 - val_acc: 0.3477
Epoch 4/100
78/78 [==============================] - 40s 518ms/step - loss: 1.6027 - acc: 0.4021 - val_loss: 1.9691 - val_acc: 0.3483
Epoch 5/100
78/78 [==============================] - 43s 554ms/step - loss: 1.5229 - acc: 0.4345 - val_loss: 1.9151 - val_acc: 0.3329
Epoch 6/100
78/78 [==============================] - 42s 539ms/step - loss: 1.4728 - acc: 0.4669 - val_loss: 1.9127 - val_acc: 0.3395
Epoch 7/100
78/78 [==============================] - 47s 600ms/step - loss: 1.3946 - acc: 0.4961 - val_loss: 1.3105 - val_acc: 0.5157
Epoch 8/100
78/78 [==============================] - 42s 535ms/s

In [ ]:
l = list(val_dataset.take(1))[0]